In [1]:
!pip install geemap pycrs earthengine-api pygis cartopy pyproj --quiet --no-cache-dir 

In [3]:
import geemap.foliumap as geemap
import ee

import matplotlib.pyplot as plt
import numpy as np

from geemap import cartoee
import cartopy.crs as ccrs
import cartopy

In [4]:
ee.Initialize(project='ee-lc-trends')

generate general variable

In [5]:
# boundaries
roi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Afghanistan'))


# satellites or sensors
L5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

# data training
training = ee.FeatureCollection('projects/ee-lc-trends/assets/Training')

# visualization parameter
visClass = {
    'min' : 0,
    'max' : 12,
    'palette' : [
        '7f776a', # Urban Area
        'd43531', # Orchards
        '9bab3e', # Intensively Irrigated Agriculture Land
        '984b34', # Irrigated Agriculture Land
        '707c44', # Rainfed Agriculture
        '76a680', # Forest
        'c08c4d', # Rangeland
        'bc4a1d', # Barren Land
        '556b7e', # Marshes
        '211f3a', # Waterbodies
        '929592', # Permanent Snow
        'FFFFFF', # Additonal-1
        '000000', # Additional-2
    ]
}

In [6]:
def scale_factors (image):
    optical_bands = image.select('SR_B.').multiply(2.75e-05).add(-0.2)
    thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

def clipping (image):
    return image.clip(roi)

# try first year

In [ ]:
image_col_90 = L5.filterDate('1990-01-01', '1990-12-31').filterBounds(roi)
image_col_90 = image_col_90.filter(ee.Filter.lt('CLOUD_COVER', 20))
image_col_90 = image_col_90.map(scale_factors)
image_col_90 = image_col_90.map(clipping)

image_90 = image_col_90.reduce(ee.Reducer.percentile([20]))
image_90 = image_90.select(['SR_B1_p20', 'SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20', 'SR_B7_p20'])
image_90 = image_90.rename(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])

ndvi = image_90.normalizedDifference(['nir', 'red']).rename('ndvi')
image_90 = image_90.addBands(ndvi)

image_90 = image_90.select(['green', 'nir', 'swir2', 'ndvi'])

image_90

adding the 5 years time series

In [ ]:
def timeseries(startY, endY, startM, endM):
    image_col = L5.filterBounds(roi)
    image_col = image_col.filter(ee.Filter.calendarRange(startY, endY, 'year')).filter(ee.Filter.calendarRange(startM, endM, 'month'))
    image_col = image_col.filter(ee.Filter.lt('CLOUD_COVER', 20))
    
    image = image_col.reduce(ee.Reducer.percentile([20]))
    image = image.clip(roi)
    image = image.multiply(2.75e-05).add(-0.2)

    image = image.select(['SR_B1_p20', 'SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20'])
    image = image.rename(['blue', 'green', 'red', 'nir', 'swir'])
    
    ndvi = image.normalizedDifference(['nir', 'red']).rename('ndvi')
    image = image.addBands(ndvi)

    image_xx = image.select(['swir', 'ndvi'])
    return ndvi

image_1 = timeseries(1988, 1992, 1, 3).rename('image_1')
image_2 = timeseries(1988, 1992, 4, 6).rename('image_2')
image_3 = timeseries(1988, 1992, 7, 9).rename('image_3')
image_4 = timeseries(1988, 1992, 10, 12).rename('image_4')

In [ ]:
image_90 = ee.Image([image_90, image_1, image_2, image_3, image_4])

image_90

In [ ]:
task = ee.batch.Export.image.toAsset(
    image = image_90,
    description = 'Image 1990',
    assetId = 'projects/ee-lc-trends/assets/image_90',
    scale = 30
)
task.start()

sampling

In [ ]:
sampled_gcp = image_90.sampleRegions(
    collection=training, scale=30, geometries=True
)

In [ ]:
sampled_gcp = sampled_gcp.randomColumn()
train_gcp = sampled_gcp.filter(ee.Filter.lt('random', 0.8))
test_gcp = sampled_gcp.filter(ee.Filter.gte('random', 0.5))

In [ ]:
trained = ee.Classifier.smileRandomForest(200).train(
    features=train_gcp,
    classProperty='landcover_',
    inputProperties=image_90.bandNames()
)
classify = image_90.classify(trained)

area limitation

In [ ]:
m = geemap.Map(center = [34.28, 66.28], zoom = 7)
m.addLayer(classify, visClass, name="Classified 1990")
m

In [ ]:
fig = plt.figure(figsize=(15, 10))

bbox = [73.33, 29.46, 59.36, 38.58]

ax = cartoee.get_map(classify, vis_params=visClass, region=bbox)
cartoee.add_gridlines(ax, interval=1, xtick_rotation=0, linestyle=":")

ax.set_title(label='LULC 1990', fontsize=15)

plt.show()

# try another year

In [ ]:
image_col_00 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterDate('2000-01-01', '2000-12-31').filterBounds(roi)
image_col_00 = image_col_90.filter(ee.Filter.lt('CLOUD_COVER', 20))

image_00 = image_col_00.reduce(ee.Reducer.percentile([20]))
image_00 = image_00.clip(roi)
image_00 = image_00.multiply(2.75e-05).add(-0.2)
image_00 = image_00.select(['SR_B1_p20', 'SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20'])

image_00

In [ ]:
sampled_gcp_00 = image_00.sampleRegions(
    collection=training, scale=30, geometries=True
)

In [ ]:
sampled_gcp_00 = sampled_gcp_00.randomColumn()
train_gcp_00 = sampled_gcp_00.filter(ee.Filter.lt('random', 0.8))
test_gcp_00 = sampled_gcp_00.filter(ee.Filter.gte('random', 0.8))

In [ ]:
trained_00 = ee.Classifier.smileRandomForest(10).train(
    features=train_gcp_00,
    classProperty='CLASS_CODE',
    inputProperties=image_00.bandNames()
)
classify_00 = image_00.classify(trained_00)

# try another year (2)

In [ ]:
image_col_10 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterDate('2010-01-01', '2010-12-31').filterBounds(roi)
image_col_10 = image_col_10.filter(ee.Filter.lt('CLOUD_COVER', 20))

image_10 = image_col_10.reduce(ee.Reducer.percentile([20]))
image_10 = image_10.clip(roi)
image_10 = image_10.multiply(2.75e-05).add(-0.2)
image_10 = image_10.select(['SR_B1_p20', 'SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20'])

image_10

In [ ]:
sampled_gcp_10 = image_10.sampleRegions(
    collection=training, scale=30, geometries=True
)

sampled_gcp_10 = sampled_gcp_10.randomColumn()
train_gcp_10 = sampled_gcp_10.filter(ee.Filter.lt('random', 0.8))
test_gcp_10 = sampled_gcp_10.filter(ee.Filter.gte('random', 0.8))

In [ ]:
trained_10 = ee.Classifier.smileRandomForest(10).train(
    features=train_gcp_10,
    classProperty='CLASS_CODE',
    inputProperties=image_10.bandNames()
)
classify_10 = image_10.classify(trained_10)

# try another year (3)

In [ ]:
image_col_20 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterDate('2020-01-01', '2020-12-31').filterBounds(roi)
image_col_20 = image_col_20.filter(ee.Filter.lt('CLOUD_COVER', 20))

image_20 = image_col_20.reduce(ee.Reducer.percentile([20]))
image_20 = image_20.clip(roi)
image_20 = image_20.multiply(2.75e-05).add(-0.2)
image_20 = image_20.select(['SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20', 'SR_B6_p20'])

image_20

In [ ]:
sampled_gcp_20 = image_20.sampleRegions(
    collection=training, scale=30, geometries=True
)

sampled_gcp_20 = sampled_gcp_20.randomColumn()
train_gcp_20 = sampled_gcp_20.filter(ee.Filter.lt('random', 0.8))
test_gcp_20 = sampled_gcp_20.filter(ee.Filter.gte('random', 0.8))

In [ ]:
trained_20 = ee.Classifier.smileRandomForest(10).train(
    features=train_gcp_20,
    classProperty='CLASS_CODE',
    inputProperties=image_20.bandNames()
)
classify_20 = image_20.classify(trained_20)

In [ ]:
from osgeo import gdal
from osgeo import osr 
import time

timedate = image_20.get('GENERATION_TIME').getInfo()
latlon = ee.Image.pixelLonLat()
latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=roi,
    maxPixels=1e10,
    scale=30,
)

data = np.array(ee.Array(latlon.get("nd")).getInfo())
lats = np.array(ee.Array(latlon.get("latitude")).getInfo())
lons = np.array(ee.Array(latlon.get("longitude")).getInfo())

unique_lats = np.unique(lats)
unique_lons = np.unique(lons)

ncols = len(unique_lons)
nrows = len(unique_lats)

ys = unique_lats[1] - unique_lats[0]
xs = unique_lons[1] - unique_lons[0]

arr = np.zeros([nrows, ncols], np.float32)

# fill array with value
counter = 0
for y in range(0, len(arr), 1):
    for x in range(0, len(arr[0]), 1):
        if lats[counter] == unique_lats[y] and lons[counter] == unique_lons[x] and counter < len(lats)-1:
            counter+=1
            arr[len(unique_lats)-1-y, x] = data[counter]

In [ ]:
m = geemap.Map(center = [34.28, 66.28], zoom = 7)
m.addLayer(classify, visClass, name="Classified 1990")
m.addLayer(classify_00, visClass, name="Classified 2000")
m.addLayer(classify_10, visClass, name="Classified 2010")
m.addLayer(classify_20, visClass, name="Classified 2020")
m

# map out over four years

In [ ]:
fig = plt.figure(figsize=(15, 10))

bbox = [73.33, 29.46, 59.36, 38.58]

ax = cartoee.get_map(classify_20, vis_params=visClass, region=bbox)
cartoee.add_gridlines(ax, interval=1, xtick_rotation=0, linestyle=":")

ax.set_title(label='LULC 2020', fontsize=15)

# plt.legend([
#     'Permanent Snow',
#     'Forest',
#     'Urban Area',
#     'Waterbodies',
#     'Irrigated Agriculture Land',
#     'Rangeland',
#     'Barren Land'
# ], )

cartoee.add_north_arrow(
    ax, text="N", xy=(0.05, 0.25), text_color="black", arrow_color="black", fontsize=20
)

# cartoee.add_scale_bar_lite(
#     ax, length=10, xy=(0.1, 0.05), fontsize=20, color="white", unit="km"
# )

plt.show()

# cartoee.savefig(fig, fname="LULC-2020.jpg", dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(24,7), 
                               subplot_kw={"projection": ccrs.Mercator()})
fig.suptitle('Comparison of LULC in Afghanistan over the decades')

bbox = [73.33, 29.46, 59.36, 38.58]

imgs = np.array([[classify, classify_00], [classify_10, classify_20]])
titles = np.array([['Year 1990', 'Year 2000'], ['Year 2010', 'Year 2020']])

def ax(i,j):
    for i in range(len(imgs)):
        for j in range(len(imgs[i])):
            ax[i,j] = cartoee.add_layer(imgs[i,j], ax=ax[i,j],
                                      region=bbox,
                                       visParams=visClass
                                      )
            ax[i,j].coastlines()
            ax[i,j].gridlines(linestyle='-')
            ax[i,j].set_title(titles[i,j])

plt.tight_layout()
cax = fig.add_axes([0.9, 0.2, 0.02, 0.6])

plt.show()

In [ ]:
lc_gt_val = geemap.shp_to_ee('./afg/afg_val.shp')

validation_class = classify.sampleRegions(lc_gt_val, scale=30)

task = ee.batch.Export.table.toDrive(validation_class.select(['lc_id', 'classification']), description='tableForValidation',fileFormat='CSV')
task.start()

In [ ]:
import pandas as pd

df = pd.read_csv('./tableForValidation.csv')

print(len(df[df['lc_id'] == df['classification']])/len(df['lc_id']))

for i1 in range(0,6+1):
  df_i1 = df[df['lc_id'] == i1]

  t_list = []
  for i2 in range(0,6+1):
    t_list.append( len(df_i1[df_i1['classification'] == i2]) )

  t_list = np.array(t_list)

  print(i1, np.round(t_list/np.sum(t_list)*100))

In [ ]:
task = ee.batch.Export.image.toDrive(
    image=classify,
    description='Classified Image',
    folder='Google Earth Engine',
    region=roi
)
task.start()